A few tests to check that brain data is getting sent to `cortex.webshow` correctly. First, try loading the raw image data.

In [3]:
import numpy as np
import cortex

In [4]:
import realtimefmri

In [5]:
from glob import glob

In [6]:
from nibabel import load as nbload, save as nbsave, Nifti1Image

# Raw activity
- Load raw PixelData files
- Convert to Nifti1 using the given `input_affine`
- Transform to reference image.

Looks good -- high intensity signal in frontal cortex.

In [18]:
data_paths = glob('../benchmark_data/pixeldata_07/*.PixelData')

In [68]:
volumes = np.empty((len(data_paths), 100, 100, 24))

In [20]:
input_affine = np.array([
        [-2.24000001, 0., 0., 108.97336578],
        [0., -2.24000001, 0., 131.32203674],
        [0., 0., 4.12999725, -59.81945419],
        [0., 0., 0., 1.]
    ])

In [21]:
funcref_nifti1 = nbload('../database/S1/funcref.nii')
raw_to_volume = realtimefmri.preprocessing.RawToVolume()

In [22]:
print len(data_paths),
for i, data_path in enumerate(data_paths):
    print i,
    with open(data_path, 'r') as f:
        raw_image_binary = f.read()
    raw_image_volume = raw_to_volume.run(raw_image_binary)['raw_image_volume']
    registered_image = realtimefmri.image_utils.transform(
        Nifti1Image(raw_image_volume, input_affine), funcref_nifti1)
    volumes[i,...] = registered_image.get_data()

75 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74


In [62]:
cortex.webshow(cortex.Volume(volumes.transpose([0,3,2,1]), 'S1', '20140802JG_avsnr_auto', vmin=0, vmax=1024))

Started server on port 1162
['rois']


<JS: window.viewers>

# Masked activity
- Load raw image data
- `wmd.get_activity_in_masks()` transforms the image to reference and then extracts voxels in the gray matter mask

Looks good -- run alongside the raw viewer to see that the data are the same

In [37]:
wmd = realtimefmri.preprocessing.WMDetrend()

gm_mask = np.empty((volumes.shape[0], wmd.masks['gm'].sum()))
print len(data_paths),
for i, data_path in enumerate(data_paths):
    print i,
    with open(data_path, 'r') as f:
        raw_image_binary = f.read()
    raw_image_volume = raw_to_volume.run(raw_image_binary)['raw_image_volume']
    gm_mask[i,:] = wmd.get_activity_in_masks(raw_image_volume)['gm']

In [66]:
cortex.webshow(cortex.Volume(gm_mask, 'S1', '20140802JG_avsnr_auto', vmin=0, vmax=1024))

Started server on port 22371
['rois']


<JS: window.viewers>

# White matter-detrended gray matter activity
- Load raw PixelData
- Run 

In [64]:
gm_detrend = np.empty((volumes.shape[0], wmd.masks['gm'].sum()))

In [65]:
print len(data_paths),
for i, data_path in enumerate(data_paths):
    print i,
    with open(data_path, 'r') as f:
        raw_image_binary = f.read()
    raw_image_volume = raw_to_volume.run(raw_image_binary)['raw_image_volume']
    gm_detrend[i, :] = wmd.run(raw_image_volume)['gm_detrend']

75 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74


In [67]:
cortex.webshow(cortex.Volume(gm_detrend, 'S1', '20140802JG_avsnr_auto'))

Started server on port 32415
['rois']


<JS: window.viewers>

Stopping server
Stopping server
